In [1]:
import os 
import sys 

import minari

import torch 
import torch.utils.data as data

from data.EpisodesDataset import EpisodesDataset

from models.cl_model import mlpCL 
from models.cmhn import cmhn 
from models.beta_model import LearnedBetaModel

from data.StatesDataset import StatesDataset
from data.TrajectorySet import TrajectorySet 
from data.Sampler import Sampler 

from trainer.beta_trainer import train_beta_model

from utils.sampling_states import sample_states 
from utils.tensor_utils import split_data

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

In [2]:
MINARI_DATASET = minari.load_dataset("D4RL/pointmaze/large-v2")
PROJECT_ROOT = os.getcwd()
CHECKPOINT_PATH = PROJECT_ROOT + "\saved_beta_models"

PROJECT_NAME = "Learning Beta Model"
RUN_NAME = "real-run 5"
FILENAME = "real_run_model"
DEVICE = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

CONFIG = {
        "num_states": 100000,  
        "lr": 1e-3,
        "weight_decay": 1e-5, 
        "masking_ratio": 0.3,
        "beta_max": 200,
        "max_epochs": 1000,
        "filename": FILENAME,
        "device": DEVICE,
    }

In [3]:
# Load cmhn model 
mhn = cmhn(update_steps=1, device=DEVICE)

# Load trained CL model 
model_name = "best_model_laplacian.ckpt"
pretrained_model_file = os.path.join(PROJECT_ROOT+ "/best models", model_name) 

if os.path.isfile(pretrained_model_file): 
    print(f"Found pretrained model at {pretrained_model_file}, loading...") 
    cl_model = mlpCL.load_from_checkpoint(pretrained_model_file, map_location=torch.device(DEVICE))

Found pretrained model at c:\Users\ray\Documents\2025 RA\contrastive-learning-RL/best models\best_model_laplacian.ckpt, loading...


In [4]:
# Preprocessing step 

data = sample_states(dataset=MINARI_DATASET, num_states=CONFIG["num_states"])
train, val = split_data(data, split_val=0.8) 

print(train.shape)
print(val.shape)

train_ds = StatesDataset(cl_model=cl_model, minari_dataset=MINARI_DATASET, data=train)
val_ds = StatesDataset(cl_model=cl_model, minari_dataset=MINARI_DATASET, data=val)

(80000, 4)
(20000, 4)


In [5]:
wandb_logger = WandbLogger(
        project=PROJECT_NAME, 
        name=RUN_NAME, 
        save_dir = PROJECT_ROOT, 
        log_model=True,
        config = CONFIG) 

In [6]:
MINIBATCH_SIZE = CONFIG["num_states"]
#print(PROJECT_ROOT)
#print(CHECKPOINT_PATH)

In [7]:
model = train_beta_model(
    bm_model=LearnedBetaModel,
    cmhn=mhn, 
    train_ds=train_ds,
    val_ds = val_ds,
    batch_size=1, 
    logger=wandb_logger, 
    checkpoint_path=CHECKPOINT_PATH, 
    max_epochs=CONFIG["max_epochs"],
    device=CONFIG["device"], 
    filename= FILENAME,

    # kwaargs
    lr=CONFIG["lr"],
    weight_decay=CONFIG["weight_decay"], 
    masking_ratio=CONFIG["masking_ratio"], 
    beta_max=CONFIG["beta_max"],
)

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 10
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: ray-s (ray-s-university-of-alberta) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\ray\Documents\2025 RA\contrastive-learning-RL\saved_beta_models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params | Mode 
------------------------------------------------
0 | dropout  | Dropout    | 0      | train
1 | beta_net | Sequential | 8.4 K  | train
2 | fc_nn    | Sequential | 4.2 K  | train
------------------------------------------------
12.6 K    Trainable params
0         Non-trainable params
12.6 K    Total params
0.050     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode


c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 2:  61%|██████    | 48440/80000 [08:11<05:20, 98.47it/s, v_num=54zi, train/nll_loss_step=0.000, train/top1_step=1.000, val/nll_loss=0.000, val/top1=1.000, train/nll_loss_epoch=0.000, train/top1_epoch=1.000]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined